In [1]:
import pyvo.dal
from pyvo.auth import authsession, securitymethods
import getpass
import requests
import pyvo

In [2]:
# To avoid warnings
import warnings
warnings.simplefilter("ignore")

In [3]:
# Url of the TAP service
tap_url = "https://archive.cefca.es/catalogues/vo/tap/jpas-idr202406"

In [4]:
# Login
user = input("Username:")
pwd = getpass.getpass("Password:")
archive_login_url = "https://archive.cefca.es/catalogues/login"
login_args = {"login": user, "password": pwd,
              "submit": "Sign+In"}
login_header = {"Content-type": "application/x-www-form-urlencoded",
                "Accept": "text/plain"}


Username: lgutierrez
Password: ········


In [5]:
pyvo.dal.tap.s = requests.Session()
response = pyvo.dal.tap.s.post(archive_login_url,
                 data=login_args, headers=login_header)

In [6]:
response.raise_for_status()

In [7]:
auth = authsession.AuthSession()
auth.credentials.set(securitymethods.ANONYMOUS, pyvo.dal.tap.s)


In [8]:
# Ejecutando la consulta
service = pyvo.dal.TAPService(tap_url, session=auth)
try:
    resultset = service.run_sync("SELECT COUNT(*) as total FROM jpas.MagABDualObj WHERE MAG_ERR_APER_6_0[jpas::J0660] < 0.4 AND MAG_ERR_APER_6_0[jpas::iSDSS] < 0.4")
except pyvo.DALQueryError as e:
    print(f"Error en consulta: {e}")

In [9]:
# Convertir a tabla Astropy
table = resultset.to_table()

In [10]:
# Obtener el número total de objetos
total_objects = table[0]["total"]
print("El total de objetos en el survey es:", total_objects)

El total de objetos en el survey es: 1792103


La tabla total tiene  12031905 objetos

### Probemos async

In [11]:
job = service.run_async("SELECT TOP 100 * FROM jpas.MagABDualObj WHERE MAG_ERR_APER_6_0[jpas::J0660] < 0.2 AND MAG_ERR_APER_6_0[jpas::iSDSS] < 0.2")


In [12]:
# Convertir a tabla de Astropy
table = job.to_table()
type(table)

astropy.table.table.Table

In [13]:
# Mostrar los primeros registros
print(table)

tile_id number    alpha_j2000     ... mag_aper_cor_6_0 mag_err_aper_cor_6_0
                      deg         ...       mag                            
------- ------ ------------------ ... ---------------- --------------------
   9499  43391  128.3961228741231 ... 26.756 .. 19.137      51.471 .. 0.011
   9499  43407  128.6162184598354 ... 24.031 .. 19.685       4.258 .. 0.015
   9499  43421 128.91236330218172 ... 22.366 .. 19.163       1.035 .. 0.009
   9499  43453 128.76709464721046 ...   22.5 .. 19.651       1.018 .. 0.012
   9499  43479 128.81820872008612 ...   99.0 .. 20.032        99.0 .. 0.019
   9499  43488 128.59589211793258 ... 20.295 .. 15.116       0.125 .. 0.001
   9499  43496   128.596269919069 ... 24.892 .. 17.607       8.478 .. 0.003
   9499  43506 128.57579650816612 ... 21.934 .. 18.487        0.61 .. 0.006
   9499  43516 128.35330475730598 ... 20.604 .. 18.576       0.179 .. 0.007
    ...    ...                ... ...              ...                  ...
   9499  450

In [14]:
for i in table.columns:
    print(i)

tile_id
number
alpha_j2000
delta_j2000
hpix11
x_image
y_image
r_eff
fwhm_world
a_world
b_world
theta_j2000
isoarea_world
relerra_world
relerrb_world
errtheta_j2000
class_star
mag_auto
mag_iso
mag_petro
mu_max
petro_radius
kron_radius
mag_aper_0_8
mag_aper_1_0
mag_aper_1_2
mag_aper_1_5
mag_aper_2_0
mag_aper_3_0
mag_aper_4_0
mag_aper_6_0
mag_err_auto
mag_err_iso
mag_err_petro
mag_err_aper_0_8
mag_err_aper_1_0
mag_err_aper_1_2
mag_err_aper_1_5
mag_err_aper_2_0
mag_err_aper_3_0
mag_err_aper_4_0
mag_err_aper_6_0
flags
norm_wmap_val
mask_flags
single_detect
mag_aper_cor_3_0
mag_err_aper_cor_3_0
mag_aper_cor_6_0
mag_err_aper_cor_6_0


# Verlos filtros

In [50]:
resultset1 = service.run_sync("SELECT * FROM  jpas.Filter")  
print(resultset1.to_table())


filter_id  name wavelength width     kx     color_representation
                    AA       nm                                 
--------- ----- ---------- ----- ---------- --------------------
        1 uJAVA       3497   509 4.91599989              #CC00FF
        2 J0378       3782   157 4.56699991              #9900FF
        3 J0390       3904   150 4.44199991              #6F008C
        4 J0400       3996   148 4.34100008              #6900B0
        5 J0410       4110   144 4.24100018              #4300DE
        6 J0420       4203   147 4.14300013              #0000FF
        7 J0430       4303   148 4.04799986              #0069FF
        8 J0440       4403   152 3.95300007              #00B8FF
        9 J0450       4503   150 3.86100006              #00FFFF
      ...   ...        ...   ...        ...                  ...
       48 J0840       8429   148 1.59500003              #97003B
       49 J0850       8523   146 1.56099999              #910034
       50 J0860       862

In [51]:
filters = resultset1.to_table()

In [53]:
for i in filters:
    print(i["name"])

uJAVA
J0378
J0390
J0400
J0410
J0420
J0430
J0440
J0450
J0460
J0470
J0480
J0490
J0500
J0510
J0520
J0530
J0540
J0550
J0560
J0570
J0580
J0590
J0600
J0610
J0620
J0630
J0640
J0650
J0660
J0670
J0680
J0690
J0700
J0710
J0720
J0730
J0740
J0750
J0760
J0770
J0780
J0790
J0800
J0810
J0820
J0830
J0840
J0850
J0860
J0870
J0880
J0890
J0900
J0910
J1007
iSDSS


## Download the entire catalog

In [43]:
query = """
SELECT TOP 10
    tile_id,
    number,
    alpha_j2000,
    delta_j2000,
    x_image,
    y_image,
    fwhm_world,
    isoarea_world,
    mu_max,
    petro_radius,
    kron_radius,
    mag_aper_cor_6_0[jpas::J0430] AS mag_J0430,  
    mag_err_aper_cor_6_0[jpas::J0430] AS err_J0430_cor,
    mag_aper_cor_6_0[jpas::J0450] AS mag_J0450,  
    mag_err_aper_cor_6_0[jpas::J0450] AS err_J0450_cor,
    mag_aper_cor_6_0[jpas::J0515] AS mag_J0515,  
    mag_err_aper_cor_6_0[jpas::J0515] AS err_J0515_cor,
    mag_aper_cor_6_0[jpas::J0660] AS mag_J0660,  
    mag_err_aper_cor_6_0[jpas::J0660] AS err_J0660_cor,
    mag_aper_6_0[jpas::iSDSS] AS mag_i,
    mag_aper_cor_6_0[jpas::iSDSS] AS mag_i_cor,
    mag_err_aper_6_0[jpas::iSDSS] AS err_i,
    mag_err_aper_cor_6_0[jpas::iSDSS] AS err_i_cor,
    mag_aper_6_0,          
    mag_aper_cor_6_0,      
    mag_err_aper_6_0,      
    mag_err_aper_cor_6_0,  
    class_star,
    flags[jpas::J0660] AS flags_J0660,  
    flags[jpas::iSDSS] AS flags_iSDSS   
FROM jpas.MagABDualObj
WHERE
    mag_err_aper_6_0[jpas::J0660] < 0.4     
    AND mag_err_aper_6_0[jpas::iSDSS] < 0.4 
    AND mask_flags[jpas::J0660] = 0        
    AND mask_flags[jpas::iSDSS] = 0         
    AND flags[jpas::J0660] <= 3            
    AND flags[jpas::iSDSS] <= 3            
    AND mag_aper_cor_6_0[jpas::iSDSS] BETWEEN 13 AND 23  
"""

In [44]:
main_data = service.run_async(query).to_table()


DALQueryError: Query Error: Unknown Enumeration Value: J0515 (at 415)

In [ ]:
main_data